In [6]:
import os
import shutil
from collections import defaultdict
import cv2
import numpy as np

# Function to resize and pad the image while maintaining aspect ratio
def resize_with_padding(image, target_size, padding_value=0):
    old_h, old_w = image.shape[:2]
    new_w, new_h = target_size
    
    # Calculate the ratio of new to old dimensions
    scale = min(new_w / old_w, new_h / old_h)
    new_w = int(old_w * scale)
    new_h = int(old_h * scale)
    
    # Resize the image to the new dimensions
    resized_image = cv2.resize(image, (new_w, new_h))
    
    # Create a new image of the target size with padding
    padded_image = np.full((target_size[1], target_size[0]), padding_value, dtype=np.uint8)
    
    # Center the resized image in the new image
    x_offset = (target_size[0] - new_w) // 2
    y_offset = (target_size[1] - new_h) // 2
    padded_image[y_offset:y_offset + new_h, x_offset:x_offset + new_w] = resized_image
    
    return padded_image

# Define the main folder path
main_folder = "."

# Step 1: Group images by their codes
image_groups = defaultdict(list)

for subfolder in os.listdir(main_folder):
    subfolder_path = os.path.join(main_folder, subfolder)
    if os.path.isdir(subfolder_path):  # Check if it's a folder
        for file_name in os.listdir(subfolder_path):
            if file_name.endswith('.png'):  # Check if the file is a PNG image
                image_code = file_name.split('_')[1].split('.')[0]  # Extract image code
                image_path = os.path.join(subfolder_path, file_name)
                image_groups[image_code].append(image_path)

# Step 2: Process image groups with more than 1 image
for image_code, image_paths in image_groups.items():
    if len(image_paths) > 1:  # Only process codes with more than 1 image
        # Create the destination folder path
        destination_folder = os.path.join(main_folder, image_code)
        if not os.path.exists(destination_folder):
            os.makedirs(destination_folder)
        
        # Move and resize/pad each image
        for image_path in image_paths:
            destination_path = os.path.join(destination_folder, os.path.basename(image_path))
            shutil.move(image_path, destination_path)
            
            # Open the image and pad it to 224x224 pixels
            image = cv2.imread(destination_path, cv2.IMREAD_GRAYSCALE)
            padded_image = resize_with_padding(image, target_size=(224, 224), padding_value=0)
            
            # Save the padded image
            cv2.imwrite(destination_path, padded_image)

print("Images have been organized, resized, and padded. Single-image codes were excluded.")


Images have been organized, resized, and padded. Single-image codes were excluded.
